In [1]:
from os.path import expanduser
import json
import pandas as pd
import geocoder
from functions.data import preprocess

with open(expanduser('~/.api/gmaps_apikey.txt')) as f: # Grab API Key
    api_key = f.readline()

In [2]:
data_dir = "data"

# Get Data

In [4]:
# Preview
turnstile_df = pd.read_csv('data/downloads/turnstile_190330.csv')
turnstile_df = preprocess(turnstile_df)
turnstile_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,00:00:00,REGULAR,6989774,2370411
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,04:00:00,REGULAR,6989795,2370413
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,08:00:00,REGULAR,6989813,2370436
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,12:00:00,REGULAR,6989924,2370512
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/23/2019,16:00:00,REGULAR,6990200,2370573


# Get Coordinates

In [34]:
station_df = pd.DataFrame(turnstile_df['station'].unique(), columns=['station'])

# Get coordinates
station_df["latlong"] = station_df["station"].apply(
    lambda x: geocoder.google(x + " Station, NY", key=api_key)
)

station_df

,station,latlong
0,34 ST-HERALD SQ,"[40.7488979, -73.98878859999999]"
1,34 ST-PENN STA,"[40.751056, -73.9903102]"
2,TIMES SQ-42 ST,"[40.7545563, -73.9866737]"
3,FLUSHING-MAIN,"[40.7595582, -73.82992209999999]"
4,59 ST,"[40.6415685, -74.0176783]"
...,...,...
205,30 AV,"[40.7668516, -73.921313]"
206,75 ST-ELDERTS,"[40.69143649999999, -73.8665512]"
207,85 ST-FOREST PK,"[40.6924861, -73.8590708]"
208,AVENUE X,"[40.5896822, -73.9742509]"


In [61]:
# format
# station_df['latlong'] = station_df['latlong'].apply(json.loads) # not sure if needed
station_df['lat'] = station_df['latlong'].apply(lambda x: x[0])
station_df['long'] = station_df['latlong'].apply(lambda x: x[1])
station_df['latlong'] = station_df['latlong'].apply(json.dumps)

# export
station_df.to_csv(f'{data_dir}/station-coordinates.csv', index=False)